In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import sys, os, re, csv, codecs

from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Dropout, Activation, Conv1D, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalMaxPool1D, Add, Flatten, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'

Using TensorFlow backend.


In [ ]:
train = pd.read_csv('drive/Datasets/Toxic Comment/train.csv', encoding='utf8')
test = pd.read_csv('drive/Datasets/Toxic Comment/test.csv', encoding='utf8')

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
train.shape

(159571, 8)

In [5]:
train.isnull().sum(),test.isnull().sum()

(id               0
 comment_text     0
 toxic            0
 severe_toxic     0
 obscene          0
 threat           0
 insult           0
 identity_hate    0
 dtype: int64, id              0
 comment_text    0
 dtype: int64)

In [ ]:
max_features = 175000  # how many unique words to use
embed_size = 300  # how big is each word vector
maxlen = 175 # max number of words in a comment to use

In [ ]:
Embedding_file = open("drive/Datasets/Toxic Comment/crawl-300d-2M.vec")

In [8]:
repl = {
    "&lt;3": " good ",
    ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}

keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)
train["new_comment_text"] = new_train_data
test["new_comment_text"] = new_test_data
print("crap removed")
trate = train["new_comment_text"].tolist()
tete = test["new_comment_text"].tolist()
for i, c in enumerate(trate):
    trate[i] = re.sub('[^a-zA-Z ?!]+', '', str(trate[i]).lower())
for i, c in enumerate(tete):
    tete[i] = re.sub('[^a-zA-Z ?!]+', '', tete[i])
train["comment_text"] = train
test["comment_text"] = test
print('only alphabets')

crap removed
only alphabets


In [9]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,new_comment_text
0,0000997932d777bf,0000997932d777bf,0,0,0,0,0,0,explanation why the edits made under my userna...
1,000103f0d9cfb60f,000103f0d9cfb60f,0,0,0,0,0,0,d'aww! he matches this background colour i am ...
2,000113f07ec002fd,000113f07ec002fd,0,0,0,0,0,0,"hey man, i am really not trying to edit war. i..."
3,0001b41b1c6bb37e,0001b41b1c6bb37e,0,0,0,0,0,0,""" more i can not make any real suggestions on ..."
4,0001d958c54c6e35,0001d958c54c6e35,0,0,0,0,0,0,"you, sir, are my hero. any chance you remember..."


In [ ]:
train = train.drop('comment_text', 1)

In [ ]:
train["comment_text"] = train["new_comment_text"]

In [ ]:
train = train.drop('new_comment_text', 1)

In [13]:
train.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text
0,0000997932d777bf,0,0,0,0,0,0,explanation why the edits made under my userna...
1,000103f0d9cfb60f,0,0,0,0,0,0,d'aww! he matches this background colour i am ...
2,000113f07ec002fd,0,0,0,0,0,0,"hey man, i am really not trying to edit war. i..."
3,0001b41b1c6bb37e,0,0,0,0,0,0,""" more i can not make any real suggestions on ..."
4,0001d958c54c6e35,0,0,0,0,0,0,"you, sir, are my hero. any chance you remember..."


In [14]:
test.head()

,id,comment_text,new_comment_text
0,00001cee341fdb12,00001cee341fdb12,yo bitch ja rule is more succesful then you'll...
1,0000247867823ef7,0000247867823ef7,"== from rfc == the title is fine as it is, imo."
2,00013b17ad220c46,00013b17ad220c46,""" == sources == * zawe ashton on lapland — / """
3,00017563c3f7919a,00017563c3f7919a,":if you have a look back at the source, the in..."
4,00017695ad8997eb,00017695ad8997eb,i do not anonymously edit articles at all.


In [ ]:
test = test.drop("comment_text", axis = 1)

In [ ]:
test.rename(columns={"new_comment_text":"comment_text"}, inplace = True)

In [17]:
test.head()

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule is more succesful then you'll...
1,0000247867823ef7,"== from rfc == the title is fine as it is, imo."
2,00013b17ad220c46,""" == sources == * zawe ashton on lapland — / """
3,00017563c3f7919a,":if you have a look back at the source, the in..."
4,00017695ad8997eb,i do not anonymously edit articles at all.


In [ ]:
list_classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
y = train[list_classes].values
train = train["comment_text"].fillna("fillna")
test = test["comment_text"].fillna("fillna")

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.1, random_state = 42, shuffle = True)

In [ ]:
raw_train = train.str.lower()
raw_test = test.str.lower()

In [ ]:
tokenizer = Tokenizer(num_words=max_features, lower = True)
tokenizer.fit_on_texts(raw_train)

train["comment_seq"] = tokenizer.texts_to_sequences(raw_train)
test["comment_seq"] = tokenizer.texts_to_sequences(raw_test)

train = pad_sequences(train.comment_seq, maxlen=maxlen)
test = pad_sequences(test.comment_seq, maxlen=maxlen)

Some of the comments are too long while some of them are too short. And we have to feed a stream of data that has a consistent length (fixed number of features).

This is why we use "Padding". We could make the shorter sentences as long as the others by filling the shortfall by zeros. But on the other hand, we also have to trim the longer ones to same length(maxlen) as the short ones. In this case, we have set the max length to be 200.

**maxlen** length depends upon: if it's too short, we might loose some useful features that could cost us some accuracy points down the path. If we put it too long, out LSTM cell will have to be larger to store the possible values or states.

One of the ways to go about it is to see the distrubution of the number of words in sentences.

In [ ]:
# Read the glove word vectors (space delimited strings) into a dictionary form word -> vector
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in Embedding_file)

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data
    
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC - AUC - epoch: %d - score: %.6f \n" % (epoch+1,score))
      
     

In [ ]:
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

In [ ]:
from keras.layers import K, Activation
from keras.engine import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Bidirectional, GRU, Flatten, SpatialDropout1D
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
rate_drop_dense = 0.28

def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

In [ ]:
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor="val_loss", verbose=1, save_best_only=True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid,y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience=5)

def get_model(lr = 0.0003, lr_d = 0.000015, units = 0, dr=0.0):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(dr)(x)
    x = Bidirectional(LSTM(units, return_sequences=True))(x)
    x = Conv1D(100, kernel_size=2, padding='valid', kernel_initializer="he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr = lr,beta_1=0.999,beta_2=0.999, epsilon=1e-08, decay = lr_d),metrics=['accuracy'])
    history = model.fit(X_train, y_train, batch_size = 128, epochs = 4, validation_data = (X_valid, y_valid), verbose=1, callbacks = [ra_val, check_point, early_stop])
    model = load_model(file_path)
  
    return model

In [33]:
model = get_model()

batch_size = 128
epochs = 3
X_tra, X_val, y_tra, y_val = train_test_split(train, y, train_size=0.9, random_state=42)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 175)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 175, 300)          52500000  
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 175, 300)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 175, 256)          329472    
_________________________________________________________________
capsule_2 (Capsule)          (None, 10, 16)            40960     
_________________________________________________________________
flatten_2 (Flatten)          (None, 160)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 160)               0         
__________

In [34]:
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=4, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/4
 86400/143613 [=================>............] - ETA: 3:48 - loss: 0.0743 - acc: 0.9774143613/143613 [==============================] - 592s 4ms/step - loss: 0.0647 - acc: 0.9791 - val_loss: 0.0440 - val_acc: 0.9836

 ROC - AUC - epoch: 1 - score: 0.977636 

Epoch 2/4
 28928/143613 [=====>........................] - ETA: 7:36 - loss: 0.0473 - acc: 0.9822143613/143613 [==============================] - 592s 4ms/step - loss: 0.0467 - acc: 0.9824 - val_loss: 0.0408 - val_acc: 0.9842

 ROC - AUC - epoch: 2 - score: 0.987736 

Epoch 3/4
 10368/143613 [=>............................] - ETA: 8:52 - loss: 0.0428 - acc: 0.9836143613/143613 [==============================] - 591s 4ms/step - loss: 0.0439 - acc: 0.9831 - val_loss: 0.0398 - val_acc: 0.9839

 ROC - AUC - epoch: 3 - score: 0.989289 

Epoch 4/4
  4352/143613 [..............................] - ETA: 9:14 - loss: 0.0396 - acc: 0.9845143613/143613 [==============================

In [36]:
y_pred = model.predict(test, batch_size=1024, verbose=1)

153164/153164 [==============================] - 50s 325us/step


In [ ]:
submission = pd.read_csv('drive/Datasets/Toxic Comment/sample_submission.csv')
submission[list_classes] = y_pred

In [ ]:
submission.to_csv('drive/Datasets/Toxic Comment/Sample.csv',index=False)